In [28]:
import wikichatter as wc
import mwapi
import requests
import json
import csv

In [29]:
#When working with mwapi, you should begin your file with a session, with a user_agent
#this allows you to make multiple calls without getting shut out of the api

session = mwapi.Session('https://en.wikipedia.org', user_agent='ewhit')

#Creating a query
>params holds the parameters you want to use when calling mwapi. This makes it easy to  
manipulate each parameter, particularly "titles" Here, we can see that titles  
has been set to ["Wikipedia:List of controversial issues"](https://en.wikipedia.org/wiki/Wikipedia:List_of_controversial_issues) 
These parameters will do the following:
    >>Tell mwapi to make a query (action), to get all of the links present (prop) on the page 
    >>"Wikipedia:List of controversial issues" (titles), and to return this in json format (format)
>The query will end up looking like this:  
    >>[https://en.wikipedia.org/w/api.php?action=query&prop=links&titles=Wikipedia:List%20of%20controversial%20issues&pllimit=max](https://en.wikipedia.org/w/api.php?action=query&prop=links&titles=Wikipedia:List%20of%20controversial%20issues&pllimit=max)

In [30]:
def params():
    return{
    "action": "query",
    "titles" : "Wikipedia:List of controversial issues",
    "prop": "links",
    "format": "json",
    "pllimit" : "1"
    }


#kwargs calls params in order to be set - this is what you will end up passing
kwargs=params()

#This holds the results of your query, and continuation=True tells session.get to keep going when it reaches the
#limit (of 50, I believe), so that you can get *all* of the links on the page
#Because we wanted all the titles on the page, query will be a generator
query = session.get(**kwargs, continuation = True)

In [31]:
#Pagelist is a list that will ultimately hold the titles of all the controversial pages
pageList = []

#This loops through all the results of the earlier query
#However, because what we want to do next is acquire the talk page for each controversial
#page, we will need to do a bit of trimming
#Ultimately, this bit of code will put "Talk:page title" into our list of pages to visit later
for request in query:
    title = json.dumps(request)
    index = title.find('links')
    temp = title[index:]
    index2 = temp.find('title')
    temp2 = temp[index2:]
    temp3 = temp2[9:]
    index3 = temp3.find('"')
    temp4 = temp3[:index3]
    toTalk = "Talk:"
    temp5 = toTalk + temp4
    pageList.append(temp5)
    

print("end for loop")

end for loop


#Queries
> Each query will return something like this: 
>> {"query": {"pages": {"27985631": {"links": [{"ns": 0, "title": "1919 World Series"}], "ns": 4, "title": "Wikipedia:List of controversial issues", "pageid": 27985631}}}, "continue": {"plcontinue": "27985631|0|1953_Iranian_coup_d'\u00e9tat", "continue": "||"}}  

>Because we want to query each individual talk page, we will need to parse the title of the talk page from what each query returns, which is what the above code does. After this loop has run, we should have a list of things that look like "Talk:1919 World Series" 

>After parsing and setting the param to the appropriate parameters, a query that looks like this should be generated:  
>>[https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&titles=Talk:1919%20World%20Series&format=json](https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&titles=Talk:1919%20World%20Series&format=json)  
>This will get the content of the page queried  
>This content is then passed to wc.parse so that it can be parsed - it looks quite unreadable initially  
>This appears to be where errors are happening

In [42]:
#This is a dictionary to hold the talk page title, and the content of that talk page
content = {}

class NoUsernameError(Exception):
    print("NUE error")

i = 0
#This loops through each page in our list of controversial talk pages
for page in pageList:
    
    #Once again, we want to define the parameters we will send to session.get
    #Here, we are again querying, and "titles" : page will query the page with the corresponding title
    #(which is why we just stored the talk page titles earlier)
    #"prop" : "revisions" gets the most recent revision, and "rvprop" : "content" gets the current content
    def param(page):
        return{
        "action": "query",
        "titles" : page,
        "prop" : "revisions",
        "rvprop" : "content",
        "format": "json",
        }
    
    #Here we again set kwargs to send to session.get
    kwargs2 = param(page)
    query = session.get(**kwargs2, continuation=True )
    
    #Because this portion of code was throwing errors, I have used several try-catch blocks
        #Several were used because the first kind returned a TypeError, so I caught that as well
    #try-catch block
    for request in query:
        #This is the text that results from the individual query (so, the talk page content)
        text = json.dumps(request)

        try:
            #Here, we try to parse the talk page content
            try:
                parsed_text = parse(text)
            except NameError:
                i = i + 1
            except NoUsernameError(parsed_text):
                i = i + 1
        except TypeError:
            i = i + 1
        except NameError:
            i = i + 1

        try:
            #Here, we add an entry to the dictionary of the current page's title and parsed text
            content[page] = json.dumps(parsed_text, indent=4)
        except NameError:
            i = i + 1
            
print("finished querrying")



NUE error
finished querrying


In [41]:
#This simply prints the page titles and their corresponding cleaned text to a file
with open("parsedTalkPages.csv", "w", newline = '') as w:
    writer = csv.writer(w, delimiter = ',')
    for key, value in content.items():
        writer.writerow([key, value])

#Output  
>The following is a line from the csv that this code generates:  

Talk:Gun Control	 

"{
    ""sections"": [
        {
            ""subsections"": [],
            ""comments"": [
                {
                    ""author"": ""130.233.179.227"",
                    ""text_blocks"": [
                        ""{\""query\"": {\""pages\"": {\""176954\"": {\""ns\"": 1, \""revisions\"": [{\""contentmodel\"": \""wikitext\"", \""*\"": \""{{Talk header}}\\n{{WikiProjectBannerShell|1=\\n{{WikiProject Maritime Trades|class=Start|importance = mid |B1=no|B2=no|B3=yes|B4=yes|B5=yes}}\\n{{WikiProject Greece|class=Start|importance=high|peer-review=yes}}\\n{{WikiProject Biography\\n|living=no\\n|class=start\\n|listas=Onassis, Aristotle\\n}}\\n}}\\n{{Vital article|level=5|topic=People|subpage=Business|class=Start}}\\n\\n==First name==\\n\\nWhy is the first name written \\\""Aristotle\\\"" in this article? I understand that there is a (centuries long) convention of writing the ancient Greek philosopher's name (incorrectly) as \\\""Aristotle\\\"", but Onassis is a different person, and his name is \\\""Aristoteles\\\"", not \\\""Aristotle\\\"".  <!-- Template:Unsigned IP --><small class=\\\""autosigned\\\"">\\u2014&nbsp;Preceding [[Wikipedia:Signatures|unsigned]] comment added by [[Special:Contributions/130.233.179.227|130.233.179.227]] ([[User talk:130.233.179.227#top|talk]]) 05:42, 10 February 2017 (UTC)</small> <!--Autosigned by SineBot-->\\n\\n==Monaco==\\nIn the 1960s there were rumours that Onassis was trying to gain control of Monaco. [[Quentin Crewe]] reported that they weren't true. They appear to have originated when Onassis gained a controlling stake in [[Soci\\u00e9t\\u00e9 des bains de mer de Monaco]]. Does anyone know any more about this? [[User:Cagliost|cagliost]] ([[User talk:Cagliost|talk]]) 16:08, 19 June 2014 (UTC)\\n\\nDraft:\\n\\n=== Monaco ===\\n\\nIn 1952, Onassis started buying shares in [[Soci\\u00e9t\\u00e9 des bains de mer de Monaco]], which had been making losses. He soon owned a controlling interest. However, Prince Rainier retained a veto over decisions. \\n\\nOnassis wanted to help develop Monaca. He had the idea that [[Prince Rainier of Monaco]] should improve the image of Monaco, then considered rather seedy, by marrying an American film star. Rainier eventually married Grace Kelly, with Onassis making a contribution towards the cost of the wedding. \\n\\nOnassis wanted to transform Monaco from a little-developed principality into a \\\""playground for the rich\\\"", but Rainier wanted a greater variety of industries. Onassis sought support from the National Council of Monaco, and there were even rumours that he was trying to take over the country. These rumours were dispelled by journalist [[Quentin Crewe]]. \\n\\nIn 1964, Rainier regained control over SBM by creating 600,000 new shares, to be controlled by the state. Within a week, Onassis left Monaco on his yacht.\\n\\n==Bisexuality==\\nThis article does not mention Onassis's bixsexuality.  I came here to read more about it.  Can somebody please put that in?  Thanks.\\n\\n== Edit of Lead paragraph - Jan 2015 ==\\n\\nthis article was tagged for needing a fuller lead paragraph.  I have edited the lead in an effort to address this and I have removed the tag.  Feel free to improve upon my efforts.  [[User:Mdukas|Mdukas]] ([[User talk:Mdukas|talk]]) 07:16, 7 January 2015 (UTC)\\n\\n== Definition of a stepchild -- does it apply to Arabella and Patrick? ==\\n\\nThe infobox lists Arabella Kennedy as stepdaughter and Patrick Kennedy as stepson.  Both died years before their mother married Onassis.  Is it accurate to call them his stepchildren?  Is a stepchild anyone whose parent one ever married?  Or is it only if the marriage occurs during the child's lifetime?[[Special:Contributions/47.139.42.69|47.139.42.69]] ([[User talk:47.139.42.69|talk]]) 07:31, 21 August 2016 (UTC)\\n\\n== External links modified ==\\n\\nHello fellow Wikipedians,\\n\\nI have just modified one external link on [[Aristotle Onassis]]. Please take a moment to review [https://en.wikipedia.org/w/index.php?diff=prev&oldid=789708100 my edit]. If you have any questions, or need the bot to ignore the links, or the page altogether, please visit [[User:Cyberpower678/FaQs#InternetArchiveBot|this simple FaQ]] for additional information. I made the following changes:\\n*Added archive https://web.archive.org/web/20070325182652/http://www.thebiographychannel.co.uk/ to http://www.thebiographychannel.co.uk/\\n\\nWhen you have finished reviewing my changes, you may follow the instructions on the template below to fix any issues with the URLs.\\n\\n{{sourcecheck|checked=false|needhelp=}}\\n\\nCheers.\\u2014[[User:InternetArchiveBot|'''<span style=\\\""color:darkgrey;font-family:monospace\\\"">InternetArchiveBot</span>''']] <span style=\\\""color:green;font-family:Rockwell\\\"">([[User talk:InternetArchiveBot|Report bug]])</span> 02:12, 9 July 2017 (UTC)\\n\\n== Mobil is Socony ==\\n\\nA quote from the article:\\n\\n'Onassis made large profits when the Big Oil companies like Mobil, Socony, and Texaco..'\\n\\nNow, 'Socony' is Standard Oil Company of New York. In 1920, the company registered the name \\\""Mobiloil\\\"" as a trademark. In 1955, Socony-Vacuum was renamed Socony Mobil Oil Company. In 1963, it changed its trade name from \\\""Mobiloil\\\"" to simply \\\""Mobil\\\"". So my point is that 'Socony' and 'Mobil' are the same thing. I'm making this little quibble, is all..I think the sentence could be rephrased. \\n\\nWhile I am at it, I'm reading here about how these boats had Panamanian flags etc. Now, of course, Aristotle Onassis was the most famous shipowner of all. And he was  at the peak of his energies and creative powers in, like, the 1950s. And of course, he was seldom out of the limelight during this period. But when was this business concerning Panamanian flags? I quote: \\n\\n'Onassis built up a fleet of freighters and tankers that eventually exceeded seventy vessels. Onassis's fleet had Panamanian flags and sailed tax-free while operating at low cost. Because of this, Onassis could turn a profit in every transaction, even though he charged one of the lowest prices in the merchant navy market. Onassis made large profits when the Big Oil companies like Mobil, Socony, and Texaco signed long-term contracts at fixed prices with him..'\\n\\nOkay, I read 'exceeded seventy vessels', when my understanding is that 'peak of 67 vessels' is accurate. Of course we need a reference, but what is the current reference? And about these Panamanian flags..there is a notion of 'flags of convenience' but I find the way that it is touched on here to be rather breezy and tendentious. Note that  one of the world\\u2019s busiest shipping lanes (one of the world's most important marine trade routes) is the Panama Canal. Being registered under Panama's flag enables ships to pass through, maybe? Note that Panama has the largest shipping fleet in the world, greater than those of the US and China combined. That's interesting, sure, but it's also not simply about Aristotle Onassis. I think it is true, that most merchant ships flying Panama's flag belong to foreign owners wishing to avoid the stricter marine regulations imposed by their own countries. About 8,600 ships fly the Panamanian flag. By comparison, the US has around 3,400 registered vessels. \\n\\nI find these parts of the article to be stated at an oddly low sort of level, like I'm reading a conspiracy theory -- let's go through some of this:\\n\\n'Onassis could turn a profit in every transaction, even though he charged one of the lowest prices in the merchant navy market...'\\n\\nOkay, is it relevant, here, that in the middle nineteen\\u2010fifties he ordered five 28,500\\u2010ton tankers at a cost of $35\\u2010million. Mr. Onassis had been told that such supertankers would never make money, as they could not negotiate the Suez Canal.  A decade later he and other shipowners were building 250,000\\u2010tonners. You see,  the closing of the canal in 1956 by President Gama Abdel Nasser of the United Arab Republic was a stroke of luck for Mr. Onassis, who made millions with his speedy supertanker hauls around the Cape of Good Hope. And where the 1967 Arab\\u2010Israeli was again closed the canal, the freight rate on oil soared from $5 to $18 a ton. By 1974, with the advent of the oil crisis, the price had almost doubled.\\n\\nI am sorry about not offering better references -- I didn't mean to put any time into this. I'm just a drive-by snarker, and no expert on the subject. [[User:DanLanglois|DanLanglois]] ([[User talk:DanLanglois|talk]]) 21:45, 30 September 2017 (UTC)\\n\\n==Birth date==\\nLede has 20 January, infobox has 15 January.  Needs fixing.  --  [[User:JackofOz|<span style=\\\""font-family: Papyrus;\\\"">Jack of Oz</span>]] [[User talk:JackofOz#top|<span style=\\\""font-size:85%; font-family: Verdana;\\\""><sup>[pleasantries]</sup></span>]] 21:48, 8 January 2018 (UTC)\"", \""contentformat\"": \""text/x-wiki\""}], \""title\"": \""Talk:Aristotle Onassis\"", \""pageid\"": 176954}}}, \""batchcomplete\"": \""\""}""
                    ],
                    ""comments"": [],
                    ""time_stamp"": ""05:42, 10 February 2017 (UTC)"",
                    ""cosigners"": [
                        {
                            ""author"": ""Cagliost"",
                            ""time_stamp"": ""16:08, 19 June 2014 (UTC)""
                        },
                        {
                            ""author"": ""Mdukas"",
                            ""time_stamp"": ""07:16, 7 January 2015 (UTC)""
                        },
                        {
                            ""author"": ""47.139.42.69"",
                            ""time_stamp"": ""07:31, 21 August 2016 (UTC)""
                        },
                        {
                            ""author"": ""InternetArchiveBot"",
                            ""time_stamp"": ""02:12, 9 July 2017 (UTC)""
                        },
                        {
                            ""author"": ""DanLanglois"",
                            ""time_stamp"": ""21:45, 30 September 2017 (UTC)""
                        },
                        {
                            ""author"": ""JackofOz"",
                            ""time_stamp"": ""21:48, 8 January 2018 (UTC)""
                        }
                    ]
                }
            ]
        }
    ]
}"
																																				

#Raw output  
>This is a line from the non-parsed output file, for comparison:

Talk:1919 World Series	{"batchcomplete": "", "query": {"pages": {"1915271": {"pageid": 1915271, "ns": 1, "title": "Talk:1919 World Series", "revisions": [{"contentformat": "text/x-wiki", "contentmodel": "wikitext", "*": "{{Talk header}}\n{{WikiProject banner shell|1=\n{{WikiProject United States|class=C|importance=Low|OH=yes|OH-importance=Mid}}\n{{WikiProject Illinois|class=C  |importance=Low}}\n{{WikiProject Chicago|class=C |importance=Mid }}\n{{WikiProject Baseball|reds=yes|importance=high |class=C }}}}\n{{On this day|date1=2005-10-09|oldid1=25124080|date2=2006-10-09|oldid2=80424092}}\n\n== Black Sox Scandal == thank you for helping me with my history project Wikipedia!\nWho is Mike Jones?? No one has ever claimed Kid Gleason (the White Sox manager) was in on the conspiracy. The first two sentences of this entry read like someone's private goof. Chick Gandil, the first baseman of the White Sox and not the manager, is widely believed to have engineered the conspiracy on the players' side.\n\n[[User:207.237.215.155]] asked this in the Black Sox section: \"does anyone have proof of this? sounds like urban legen\nd to me\". [[User:Stephen Compall|Stephen Compall]] 15:04, 20 May 2005 (UTC)\n\nIf what you are asking about is where the term 'Black Sox' comes from, the truth and the myth are quite intermixed at this point.  Comiskey was certainly known as very tight man about money, and regularly he would do anything in his power to short his players.  The most well known incident of this is when Chicotte, the Sox's ace, was benched (under Comiskey's orders)in 1919 for the final two plus weeks of the season to prevent him from reaching 30 wins which would have kicked in a $10,000 bonus - he ended the season with 29 wins and a $6,000 base salary. The stories of Comiskey forcing his players to pay for their own laundry expenses are true (he commonly forced players to pay many expenses out of their own pockets while on the road), but the 'Black Sox' title does not (as far as I know of) appear until after the 1919 series when Chicago reporters (namely those of the Chicago Herald) began to dig up dirt on the scandal (no pun intended).\n\nThis should be merged into the 1919 World Series article, as it doesn't seem to serve an independent purpose. It doesn't explain any of the reasons why the Chicago team had been considered cursed. [[User:160.227.21.41|160.227.21.41]] 01:05, 28 October 2005 (UTC)\n\nFair enough.  I'll change the page I created into a redirect ASAP.[[User:Amchow78|Amchow78]] 20:06, 28 October 2005 (UTC)\n\nSome years ago I read an article (I forget where) about the scandal by someone who studied contemporary newspaper accounts of the games and came to the conclusion that BOTH teams were trying to throw the Series (presumably on behalf of two different parties), but the White Sox simply were more successful at it.[[User:Alloco1|Alloco1]] 22:18, 19 March 2006 (UTC)\n\nIf you read \"The Glory of Their Times\" by Lawrence Ritter, the account given by Ed Roush, the star centerfielder for the Reds, states that in fact some members of the Reds were approached by gamblers to have them throw some games (including Game 3, which they won).  It is a very interesting account, since I had never read anything regarding a possible fix involving players on the Cincinnati Reds team.  But Ed Roush's account, while short, is very detailed and believeable.  By the way, I highly recommend reading not only the section on Ed Roush, but the whole book; any true fan of the game will find it an extremely pleasurable read - T.J. Swartz\n\n== date of start of curse ==\n\nI reverted the change of the start of the curse from 1917 back to 1919.  The \nidea as I've always understood it is that if it hadn't been for the sellout, they ''would'' have won in 1919.  Thus the curse starts at the moment the deal was struck.  They were not cursed in 1918, they just weren't good enough to win the pennant.  If you must revert, go ahead--it's hardly a big deal.  [[User:Chick Bowen|Chick Bowen]] 00:40, 3 November 2005 (UTC)\n\nI see your point.  As fair as it is, I used the \"Curse of the Bambino\" page as a guide for creating the timeline of the Black Sox curse.  In that page, they've put the start of the curse in 1918 ... despite the fact that Babe Ruth was sold to the Yankees in 1920 ... two years after the Red Sox last won the World Series with him. Similarily, I tried following that guideline when I started the date of the Black Sox curse.  I'll leave it @ 1919 for now ... and pose this question on the \"Curse of the Bambino\" discussion page if it hasn't been posed already. [[User:Amchow78|Amchow78]] 03:41, 10 November 2005 (UTC)\n\nAfter posing the question on the \"Curse of the Bambino\" discussion page, they've changed the start date of their curse from 1918 to 1920.  With that as a precedent setter, it looks like 1919 will stick as the year that the Curse of the Black Sox started. [[User:Amchow78|Amchow78]] 01:24, 11 November 2005 (UTC)\n\nIt should be noted, however, that the White Sox won 88 games in 1919, the Reds 95, a seven-game difference.  Also that [[Edd Roush]], the Reds' Hall-of-Fame right fielder and (I think) the last surviving player from this Series on either side, swore until his death in [[1988]] that the Reds would still have won the Series had the whole thing been on the level.  That could have just been pride in his team, but he was the last man involved who had some understanding of what was really going on -- on the field, anyway.  -- Pacholeknbnj, 2:00 PM EST, January 28, 2006\n\nWho is Mike Jones? No one has ever claimed Kid Gleason was in on the conspiracy. The first two sentences of this entry read like someone's private goof. Chick Gandil, the first baseman of the White Sox and not the manager, is widely believed to have engineered the conspiracy on the players' side.\n\n------------------\n\nI deleted the curse section.  No Sox fan talked about a curse.  Even the Chicago media never acknowledged a curse.  Sox fans never believed in one.  We just believed that we had crappy management.  Curses are excuses for idiots like Red Sox and Cub fans.\n\nI think the curse section should be left in, and I've restored it. I'm sure very few people truly believe there's some sort of curse affecting the White Sox, Red Sox, or Cubs, but such stories and superstitions are part of baseball and deserve a mention. Just because there's no actual curse doesn't mean there's no Curse story. [[User:PaulGS|PaulGS]] 18:53, 30 June 2006 (UTC)\n\n== End of the Black Sox? ==\nWhat about a section detailing the 2005 World Series, and the presumed end of the \"curse?\" [[User:Realbrvhrt|Realbrvhrt]] 20:31, 6 June 2006 (UTC)\n:Well, it's covered now -[[User:Elmer Clark|Elmer Clark]] 06:32, 18 August 2006 (UTC)\n\n==The Curse==\nWhat exactly is the basis for having a section on the \"curse\".  The information included appears to be more trivial in nature.  I would suggest that the section gets moved to trivia and can be summed up in one sentence.  A full description of what happened in the games does not seem beneficial.[[User:Tecmobowl|Tecmobowl]] 21:15, 4 October 2006 (UTC)\n\n:It's part of the lore, just like so many other superstitions in baseball. It's something hardly anyone takes seriously, but baseball, more so than other sports, is about its history, and the supposed curse is part of that. [[User:PaulGS|PaulGS]] 10:02, 4 March 2007 (UTC)\n\n::I've added this section back yet again, since I still see no reason for removing it. Just because it isn't something to be taken seriously doesn't mean it doesn't belong. [[User:PaulGS|PaulGS]] 05:25, 20 April 2007 (UTC)\n\n::There is already an article on the Black Sox Scandal.  This is about the World Series, not the circumstances surrounding it.  If you want to talk about adding content like this, please try and get some input from others before doing so. [[User:Tecmobowl|Tecmobowl]] 15:27, 21 April 2007 (UTC)\n\n:::I didn't write the section. It was there for a while, got removed (see the discussion above), and then I restored it. Maybe [[Black Sox Scandal]] is a better place for it. I could suggest getting input from others before removing it, but nobody else has commented on the section in a while. [[User:PaulGS|PaulGS]] 02:10, 23 April 2007 (UTC)\n\n== Merge==\nI see no reason to have separate articles for the Series and the Black Sox Scandal. [[User:Vidor|Vidor]] 09:01, 16 November 2006 (UTC)\n\n:I disagree wholeheartedly.  The Scandal and the series are not one in the same.  The article about the [[1919 World Series]] is to communicate the events of the series.  The article about the [[Black Sox Scandal]] is to communicate the events and experiences surrounding the series.  While there is some over lap, these are two separate issues. Similarly, there is an article for [[The Catch]] and another article for the [[1954 World Series]]. // [[User:Tecmobowl|Tecmobowl]] 09:34, 16 November 2006 (UTC)\n\n*'''Strongly oppose''': [[User:Tecmobowl|Tecmobowl]] is absolutely correct. In addition to what he said above, thre is a whole series of articles on {year} World Series. If this page were merged into the Black Sox Scandal page, should the template that connects all of the World Series pages be changed for this one series? Should the 1998 Series page be merged into the earthquake page (I think that was the year, wasn't it?)? --[[User:Ohms law|Ohms law]] 17:08, 17 November 2006 (UTC)\n\n*'''Oppose''', per Tecmobowl.  And it's [[1989 World Series|1989]] :P . [[User:Jpers36|Jpers36]] 17:44, 17 November 2006 (UTC)\n\n::Y'all are nuts.  First, the articles should be merged under THIS article, the article for the 1919 World Series.  I am not proposing removing this article from the list of World Series articles; that would be silly.  (Removed the \"merge\" recommendation from this article, to reflect that.) The scandal and the Series are the same; the scandal was about fixing the Series, the scandal as executed was the fix of the Series, and the aftermath of the scandal was the aftermath of the Series.  I entered \"1919 World Series\" into the search engine the other day, and was amazed to see very little about the fix--just a dry list of box scores, essentially.  [[User:Vidor|Vidor]] 19:02, 17 November 2006 (UTC)\n\n::Please review [[WP:NPA]] before making any comments on the topic at hand. // [[User:Tecmobowl|Tecmobowl]] 01:06, 18 November 2006 (UTC)\n\n::*Well, that makes more sense. The confusion comes from how you listed the merge request. There is a way to say \"another page should be merged to this one\", like this: {{tlp|mergefrom|SOURCE PAGE}} would be placed on this page, and {{tlp|mergeto|DESTINATION PAGE}} would be placed on the other page (Black Sox scandal, in this instance). I still think it's a bad idea though. Each topic is distinct in it's own right. There should be a short sub-section here with a paragraph about the Black Sox scandal, and a link to the full article, though. --[[User:Ohms law|Ohms law]] 08:21, 18 November 2006 (UTC)\n\n:*'''Tecmo Banned Indefinitely.'''  FYI--Tecmo has been banned indefinitely for repeated violations of Wiki policy.--[[User:Epeefleche|Epeefleche]] 01:30, 14 July 2007 (UTC)\n\n== World Series Template ==\n\nEarlier, I added the (aparently) newer {{tl|WorldSeriesRt}} picker, and hid the older {{tl|WorldSeries}} picker. Information on the newer picker's page indicated that it matches the form of other pickers, and I tend to agree with the author {{User|Schweiwikist}} that it looks better. I added the {{tl|WorldSeriesRt}} to many other world series pages, but I'll hold of on changing any more if the change is controversial at all and is just going to be reverted. All the pages should be the same is all, so which picker should we be using? --[[User:Ohms law|Ohms law]] 08:29, 18 November 2006 (UTC)\n\n: Ah, i must have missed that when i was looking in the edit history.  Sorry!  My personal preference would be to reserve the top of the article for information about to the specific world series.  I would think an infobox about the series itself, or some imagery relating to the series would be the best thing to put up in the introduction area.  If you feel strongly that the template should be placed at he top, please feel free to put it back. // [[User:Tecmobowl|Tecmobowl]] 08:39, 18 November 2006 (UTC)\n:*Well, I agree about the top ought to be being used for info specific to the series. The infobox does go along the side though, so that the top and right should still be available for imagry. Infobox's tend to be pretty cooperative, from what i've seen elsewhere. Anyway, i'll go ahead and put it back. Just didn't want to start any sort of \"edit war\", you know? --[[User:Ohms law|Ohms law]] 09:10, 18 November 2006 (UTC)\n:*Right now there is no alternative template so i can't really complain.  Certainly the fact that you started the subject here shows you have no interest in an edit war.  Let's see how this develops and go from there. // [[User:Tecmobowl|Tecmobowl]] 10:52, 18 November 2006 (UTC)\n::*Actually, in regards to alternatives. If you or anyone else knows anything about CSS, I was attempint to copy the system used on the {{tl|MLBAllStarGame}} template so that the infobox could be hidden. I didn't have any luck getting it to look right though, so if you or anyone else could lend assistance it'd be great. --[[User:Ohms law|Ohms law]] 14:21, 19 November 2006 (UTC)\n::*I'll look at it when i get a chance.  I'll see if I can't help you out.  I know css and such, but i've never touched a template on here. // [[User:Tecmobowl|Tecmobowl]] 16:38, 19 November 2006 (UTC)\n== WikiProject class rating==\nThis article was automatically assessed because at least one WikiProject had rated the article as start, and the rating on other projects was brought up to start class. [[User:BetacommandBot|BetacommandBot]] 14:20, 9 November 2007 (UTC)\n\n== Who is Burns? ==\n\nArticle makes reference 3 times to someone named \"Burns\" but there is no explanation on who this person is or even their full name.\n\n==Baseball assessment comment==\n{{Substituted comment|page=Talk:1919 World Series/CommentsBB|length=129|lastedit=20070422200432|comment=With inline citations and a little expansion, this could be B-class - [[User:Mattingly23|Mattingly23]] 20:04, 22 April 2007 (UTC)}}\nSubstituted at 14:12, 10 October 2016 (UTC)\n\n== Issues with second paragraph ==\n\nThere are some issues with statements made in the second paragraph of this article. One sentence reads \"The 1919 World Series was the last World Series to take place without a Commissioner of Baseball in place.\" However, the Wikipedia \"Kenesaw Mountain Landis\" article states he became commissioner on November 12, 1920. Obviously this took place after the 1920 World Series was played. Another sentence states \"In August 1920, seven players from the White Sox were banned from organized baseball for fixing the series (or having knowledge about the fix).\" According to the Wikipedia \"Black Sox Scandal\" article eight players (not seven) were banned by Commissioner Landis on August 3, 1921 (not August 1920). [[User:Bunkyray5|Bunkyray5]] ([[User talk:Bunkyray5|talk]]) 01:28, 4 April 2017 (UTC)\n\n== External links modified ==\n\nHello fellow Wikipedians,\n\nI have just modified one external link on [[1919 World Series]]. Please take a moment to review [https://en.wikipedia.org/w/index.php?diff=prev&oldid=785642256 my edit]. If you have any questions, or need the bot to ignore the links, or the page altogether, please visit [[User:Cyberpower678/FaQs#InternetArchiveBot|this simple FaQ]] for additional information. I made the following changes:\n*Added archive https://web.archive.org/web/20140815042229/http://www.chicagohs.org/history/blacksox.html to http://www.chicagohs.org/history/blacksox.html\n\nWhen you have finished reviewing my changes, you may follow the instructions on the template below to fix any issues with the URLs.\n\n{{sourcecheck|checked=false|needhelp=}}\n\nCheers.\u2014[[User:InternetArchiveBot|'''<span style=\"color:darkgrey;font-family:monospace\">InternetArchiveBot</span>''']] <span style=\"color:green;font-family:Rockwell\">([[User talk:InternetArchiveBot|Report bug]])</span> 16:56, 14 June 2017 (UTC)"}]}}}}																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																																
